In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from google.colab import drive
import warnings
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
warnings.filterwarnings("ignore")

In [3]:
#Define path to read the data
trading_pairs = {'JBF':'3443','QWF':'2388','HCF':'2498','DBF':'2610','EHF':'1319','IPF':'3035','IIF':'3006','QXF':'2615','PEF':'5425','NAF':'3105'}
stock_path = "/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/stocks"
stock_files = os.listdir(stock_path)
futures_path = "/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/futuresOB"
futures_files = os.listdir(futures_path)
futures_trade_path = "/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/futuresTrades"
futures_trade_files = os.listdir(futures_trade_path)
stock_files.remove('.DS_Store')
futures_files.remove('.DS_Store')
#futures_trade_files.remove('.DS_Store')

In [44]:
futures_trade_files

['HCF', 'IPF', 'IIF', 'QXF', 'QWF', 'EHF', 'NAF', 'JBF', 'PEF', 'DBF']

In [5]:
futures_files

['JBF', 'PEF', 'QXF', 'IPF', 'EHF', 'IIF', 'NAF', 'QWF', 'DBF', 'HCF']

In [4]:
def ln_ratio(x):
    return np.log(x.max() / x.min())

def count_nonzero(x):
    return x.notna().sum()

#Function to read stock data from csv file, stock name is element in stock_files
def read_stock_data(stock_name):
    #stock_data stores list of .gz files
    stock_data = os.listdir(stock_path + '/' + stock_name)
    stock_price_one_minute = pd.DataFrame()
    for file in stock_data:
      stock_tick = pd.read_csv(stock_path + '/' + stock_name + '/' + file, compression='gzip')

      #Data cleaning, drop na data.
      stock_tick= stock_tick.loc[stock_tick['SP1'] * stock_tick['BP1'] != 0]

      #Convert date to datetime format
      stock_tick['date'] = pd.to_datetime(stock_tick['date'], format="%Y-%m-%d")

      # set a timestamp every 1 min starting from 9:01, which represents trade happens in the minute just passed.
      stock_tick['new_time'] =  stock_tick['time'] +(100000 - stock_tick['time'] % 100000)

      # if minute is 60, adjust time accordingly.
      stock_tick['new_time'] = stock_tick['new_time'] + (stock_tick['new_time'] % 10000000 == 6000000) * 4000000
      stock_tick['time_stamp'] = pd.to_datetime(stock_tick.date.astype(str) + ' ' + stock_tick.new_time.astype(str), format="%Y-%m-%d %H%M%S%f")

      # get the column 'SP1','BP1' and calculate the mid quote
      stock_tick['mid_quote'] = np.log(stock_tick[['SP1','BP1']].mean(axis=1))

      result = stock_tick.groupby('time_stamp').agg({
          'mid_quote': [ln_ratio,'mean'],
          'volume': 'count',
          'size' : count_nonzero,
          'BV1': 'sum',
          'BV2': 'sum',
          'BV3': 'sum',
          'BV4': 'sum',
          'BV5': 'sum',
          'SV1': 'sum',
          'SV2': 'sum',
          'SV3': 'sum',
          'SV4': 'sum',
          'SV5': 'sum'
      })

      result['bid_ask_ratio'] = (result['BV1'] + result['BV2'] + result['BV3'] + result['BV4']+ result['BV5']) / (result['SV1'] + result['SV2'] + result['SV3'] + result['SV4'] + result['SV5'])
      result['bid1_ask1_ratio'] = (result['BV1']) / (result['SV1'])
      result = result.droplevel(0, axis=1)
      result = result.iloc[:, [0, 1, 2, 3, 14, 15]]
      result.columns = ['max_min_ratio', 'mid_quote','trade_count', 'quote_count',  'bid_ask_ratio', 'bid1_ask1_ratio']

      stock_price_one_minute = pd.concat([stock_price_one_minute, result], axis=0)
    return(stock_price_one_minute)

In [5]:
#Function to read futures data from csv file, futures name is element in futures_files
def read_futures_data(futures_name):
    #stock_data stores list of .gz files
    futures_data = os.listdir(futures_path + '/' + futures_name)
    futures_price_one_minute = pd.DataFrame()
    for file in futures_data:
      futures_tick = pd.read_csv(futures_path + '/' + futures_name + '/' + file)

      #Data cleaning, drop na data.
      futures_tick = futures_tick.loc[futures_tick['askPrice1'] * futures_tick['bidPrice1'] != 0]

      #Convert date to datetime format
      futures_tick['date'] = pd.to_datetime(futures_tick['date'], format="%Y-%m-%d")

      # set a timestamp every 1 min starting from 9:01, which represents trade happens in the minute just passed.
      futures_tick['new_time'] =  futures_tick['time'] +(100000 - futures_tick['time'] % 100000)

      # if minute is 60, adjust time accordingly.
      futures_tick['new_time'] = futures_tick['new_time'] + (futures_tick['new_time'] % 10000000 == 6000000) * 4000000
      futures_tick['time_stamp'] = pd.to_datetime(futures_tick.date.astype(str) + ' ' + futures_tick.new_time.astype(str), format="%Y-%m-%d %H%M%S%f")

      # get the column 'SP1','BP1' and calculate the mid quote
      futures_tick['mid_quote'] = np.log(futures_tick[['askPrice1','bidPrice1']].mean(axis=1))


      result = futures_tick.groupby('time_stamp').agg({
      'mid_quote': [ln_ratio,'mean'],
      'time': 'count',
      'bidSize1': 'sum',
      'bidSize2': 'sum',
      'bidSize3': 'sum',
      'bidSize4': 'sum',
      'bidSize5': 'sum',
      'askSize1': 'sum',
      'askSize2': 'sum',
      'askSize3': 'sum',
      'askSize4': 'sum',
      'askSize5': 'sum'
    })

      result['bid_ask_ratio'] = (result['bidSize1'] + result['bidSize2'] + result['bidSize3'] + result['bidSize4']+ result['bidSize5']) / (result['askSize1'] + result['askSize2'] + result['askSize3'] + result['askSize4'] + result['askSize5'])
      result['bid1_ask1_ratio'] = (result['bidSize1']) / (result['askSize1'])
      result = result.droplevel(0, axis=1)
      result = result.iloc[:, [0, 1, 2, 13, 14]]
      result.columns = ['max_min_ratio', 'mid_quote', 'quote_count',  'bid_ask_ratio', 'bid1_ask1_ratio']

      futures_price_one_minute = pd.concat([futures_price_one_minute, result], axis=0)

    return(futures_price_one_minute)


In [6]:
#Function to read futures trade data from csv file, futures name is element in futures_files
def read_trade_data(futures_name):
    #stock_data stores list of .gz files
    futures_data = os.listdir(futures_trade_path + '/' + futures_name)
    futures_trade_one_minute = pd.DataFrame()
    for file in futures_data:
      futures_tick = pd.read_csv(futures_trade_path + '/' + futures_name + '/' + file)
      #Data cleaning, drop na data.
      futures_tick = futures_tick.loc[futures_tick['totalMatchValue'] != 0]

      #Convert date to datetime format
      futures_tick['date'] = pd.to_datetime(futures_tick['Date'], format="%Y-%m-%d")

      # set a timestamp every 1 min starting from 9:01, which represents trade happens in the minute just passed.
      futures_tick['new_time'] =  futures_tick['Time'] +(100000 - futures_tick['Time'] % 100000)
      futures_tick = futures_tick.sort_values(by = ['date', 'Time'], ascending=True)
      # if minute is 60, adjust time accordingly.
      futures_tick['new_time'] = futures_tick['new_time'] + (futures_tick['new_time'] % 10000000 == 6000000) * 4000000
      futures_tick['time_stamp'] = pd.to_datetime(futures_tick.date.astype(str) + ' ' + futures_tick.new_time.astype(str), format="%Y-%m-%d %H%M%S%f")
      result = futures_tick.groupby('time_stamp').agg({
            'totalMatchSize': 'count'
          })
      result.columns = ['trade_count']

      futures_trade_one_minute = pd.concat([futures_trade_one_minute, result], axis=0)

    return(futures_trade_one_minute)


In [11]:
def calculate_spread(futures_name):
    spread = (read_stock_data(trading_pairs[futures_name]) - read_futures_data(futures_name)).dropna()
    spread = spread.reset_index()
    spread['date'] = spread['time_stamp'].dt.date
    spread['time'] = spread['time_stamp'].dt.time
    spread = spread.set_index('time_stamp', drop = True)
    return spread

In [7]:
trading_pairs.keys()

dict_keys(['JBF', 'QWF', 'HCF', 'DBF', 'EHF', 'IPF', 'IIF', 'QXF', 'PEF', 'NAF'])

In [8]:
trading_pairs['JBF']

'3443'

In [9]:
for future_name in tqdm(trading_pairs.keys()):
  stock_factor = read_stock_data(trading_pairs[future_name])
  futures_ = read_futures_data(future_name)
  futures_trade = read_trade_data(future_name).sort_index()
  stock_factor.to_csv('/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/FactorData/stocks/' + trading_pairs[future_name] + '.csv')
  futures_factor = pd.merge(futures_,futures_trade,on = 'time_stamp',how = 'left').fillna(0)
  futures_factor = futures_factor.loc[futures_trade.index[0]: ,]
  futures_factor.to_csv('/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/FactorData/futures/' + future_name + '.csv')
  spread = (stock_factor['mid_quote'] - futures_factor['mid_quote']).dropna()
  spread.to_csv('/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/NewSpread/' + future_name + '.csv')

100%|██████████| 10/10 [42:27<00:00, 254.79s/it]


In [120]:
spread = (stock_factor['mid_quote'] - futures_factor['mid_quote']).dropna()
spread.to_csv('/content/drive/MyDrive/Colab Notebooks/Independent project_2023 Summer/NewSpread/' + 'JBF.csv')